In [1]:
import os
import pandas as pd
import geopandas as gpd 
from shapely.geometry import Point
import numpy as np

In [7]:
osm_building = './raw/osm_impots.geojson'
osm_building_gdf=gpd.read_file(osm_building)
osm_building_gdf.head()

,osm_id,code,name,geometry
0,26617684,1500,Direction des impôts,POINT (4.03655 36.71474)
1,153107861,1500,Receveur des impots - Dely brahim,POINT (2.98892 36.74876)
2,185615561,1500,Lagr.ilot 24-1 Les impôts,POINT (5.41145 36.18817)
3,234050287,1500,Inspection des Impots مفتشية الضرائب وادي النجاء,POINT (6.11954 36.42813)
4,240332545,1500,Service des Impôts,POINT (3.03324 36.73448)


In [8]:
# Load the targeted dataset
impots_gdf = gpd.read_file('./impots_to_check.geojson')
impots_gdf.head()

,address,lat,lon,tel,name,wilaya,to_check,code,geometry
0,"ADRAR,Boulevard Mohamed El Atchane,Adrar",28.017440,-0.264250,049 36 41 18 - 049 36 41 23 - 049 36 41 24 ...,. IMPOTS Direction,Adrar,False,1,POINT (-0.26425 28.01744)
1,"ADRAR,Rue Taleb Mohamed,Adrar",27.876660,-0.270800,49361062,. IMPOTS Direction,Adrar,False,1,POINT (-0.27080 27.87666)
2,"ADRAR,Route 5 Juillet 1962,Adrar",28.033886,1.659626,49364129,. IMPOTS Direction,Adrar,False,1,POINT (1.65963 28.03389)
3,"ADRAR,Adrar",27.876660,-0.270800,49364113,. IMPOTS Direction,Adrar,False,1,POINT (-0.27080 27.87666)
4,"3,REGGANE,Cité El Kaous Labyad,Adrar",26.724265,0.165729,49320202,. IMPOTS Direction,Adrar,False,1,POINT (0.16573 26.72427)


In [9]:
# Define source CRS to geographic and project to a Pseudo-Mercator
# Useful for spatial analysis (buffer, intersect, ...)

impots_gdf = impots_gdf.set_crs("EPSG:4326")
impots_gdf= impots_gdf.to_crs("EPSG:3857")

osm_building_gdf = osm_building_gdf.set_crs("EPSG:4326")
osm_building_gdf = osm_building_gdf.to_crs("EPSG:3857")

osm_building_gdf.crs

<Projected CRS: EPSG:3857>
Name: WGS 84 / Pseudo-Mercator
Axis Info [cartesian]:
- X[east]: Easting (metre)
- Y[north]: Northing (metre)
Area of Use:
- name: World between 85.06°S and 85.06°N.
- bounds: (-180.0, -85.06, 180.0, 85.06)
Coordinate Operation:
- name: Popular Visualisation Pseudo-Mercator
- method: Popular Visualisation Pseudo Mercator
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [10]:
impots_gdf.reset_index(inplace=True)
impots_gdf = impots_gdf.rename(columns = {'index':'num'})

,num,address,lat,lon,tel,name,wilaya,to_check,code,geometry
0,0,"ADRAR,Boulevard Mohamed El Atchane,Adrar",28.017440,-0.264250,049 36 41 18 - 049 36 41 23 - 049 36 41 24 ...,. IMPOTS Direction,Adrar,False,1,POINT (-29416.142 3251172.790)
1,1,"ADRAR,Rue Taleb Mohamed,Adrar",27.876660,-0.270800,49361062,. IMPOTS Direction,Adrar,False,1,POINT (-30145.296 3233432.306)
2,2,"ADRAR,Route 5 Juillet 1962,Adrar",28.033886,1.659626,49364129,. IMPOTS Direction,Adrar,False,1,POINT (184748.721 3253246.711)
3,3,"ADRAR,Adrar",27.876660,-0.270800,49364113,. IMPOTS Direction,Adrar,False,1,POINT (-30145.296 3233432.306)
4,4,"3,REGGANE,Cité El Kaous Labyad,Adrar",26.724265,0.165729,49320202,. IMPOTS Direction,Adrar,False,1,POINT (18448.857 3089064.389)
...,...,...,...,...,...,...,...,...,...,...
892,892,"RELIZANE,Boulevard De L'ALN,Relizane",35.872127,1.107932,046 76 92 56 - 046 76 92 57,. IMPOTS,Relizane,False,48,POINT (123334.382 4283040.411)
893,893,"RELIZANE,Boulevard Larbi Tebessi,Relizane",35.745026,0.557884,046 71 54 09 - 046 71 57 68 - 046 71 60 00,. IMPOTS,Relizane,False,48,POINT (62103.329 4265593.864)
894,894,"RELIZANE,Cité Zeraia,Relizane",35.921380,0.069458,46769201,. IMPOTS,Relizane,True,48,POINT (7731.996 4289808.836)
895,895,"RELIZANE,Diar El Ward,Relizane",35.745026,0.557884,046 71 54 07 - 046 76 03 11 - 046 76 03 41,. IMPOTS,Relizane,False,48,POINT (62103.329 4265593.864)


In [14]:
impots_gdf['buffer'] = impots_gdf.geometry.buffer(500)
buffers = impots_gdf[['num', 'buffer']]
buffers = buffers.rename(columns={'buffer': 'geometry'})
buffers = gpd.GeoDataFrame(buffers)
buffers = buffers.set_crs("EPSG:3857")
gpd.GeoDataFrame(buffers).to_file("buffers.geojson", driver='GeoJSON')
buffers.head()

,num,geometry
0,0,"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
1,1,"POLYGON ((-29645.296 3233432.306, -29647.703 3..."
2,2,"POLYGON ((185248.721 3253246.711, 185246.314 3..."
3,3,"POLYGON ((-29645.296 3233432.306, -29647.703 3..."
4,4,"POLYGON ((18948.857 3089064.389, 18946.449 308..."


,num,address,lat,lon,tel,name,wilaya,to_check,code,geometry,buffer
0,0,"ADRAR,Boulevard Mohamed El Atchane,Adrar",28.017440,-0.264250,049 36 41 18 - 049 36 41 23 - 049 36 41 24 ...,. IMPOTS Direction,Adrar,False,1,POINT (-29416.142 3251172.790),"POLYGON ((-28916.142 3251172.790, -28918.550 3..."
1,1,"ADRAR,Rue Taleb Mohamed,Adrar",27.876660,-0.270800,49361062,. IMPOTS Direction,Adrar,False,1,POINT (-30145.296 3233432.306),"POLYGON ((-29645.296 3233432.306, -29647.703 3..."
2,2,"ADRAR,Route 5 Juillet 1962,Adrar",28.033886,1.659626,49364129,. IMPOTS Direction,Adrar,False,1,POINT (184748.721 3253246.711),"POLYGON ((185248.721 3253246.711, 185246.314 3..."
3,3,"ADRAR,Adrar",27.876660,-0.270800,49364113,. IMPOTS Direction,Adrar,False,1,POINT (-30145.296 3233432.306),"POLYGON ((-29645.296 3233432.306, -29647.703 3..."
4,4,"3,REGGANE,Cité El Kaous Labyad,Adrar",26.724265,0.165729,49320202,. IMPOTS Direction,Adrar,False,1,POINT (18448.857 3089064.389),"POLYGON ((18948.857 3089064.389, 18946.449 308..."
...,...,...,...,...,...,...,...,...,...,...,...
892,892,"RELIZANE,Boulevard De L'ALN,Relizane",35.872127,1.107932,046 76 92 56 - 046 76 92 57,. IMPOTS,Relizane,False,48,POINT (123334.382 4283040.411),"POLYGON ((123834.382 4283040.411, 123831.974 4..."
893,893,"RELIZANE,Boulevard Larbi Tebessi,Relizane",35.745026,0.557884,046 71 54 09 - 046 71 57 68 - 046 71 60 00,. IMPOTS,Relizane,False,48,POINT (62103.329 4265593.864),"POLYGON ((62603.329 4265593.864, 62600.922 426..."
894,894,"RELIZANE,Cité Zeraia,Relizane",35.921380,0.069458,46769201,. IMPOTS,Relizane,True,48,POINT (7731.996 4289808.836),"POLYGON ((8231.996 4289808.836, 8229.588 42897..."
895,895,"RELIZANE,Diar El Ward,Relizane",35.745026,0.557884,046 71 54 07 - 046 76 03 11 - 046 76 03 41,. IMPOTS,Relizane,False,48,POINT (62103.329 4265593.864),"POLYGON ((62603.329 4265593.864, 62600.922 426..."
